## Data Collection

In [ ]:
!wget https://github.com/nadamakram/nasa-sac22/raw/main/temperature-data.zip

In [ ]:
!unzip /content/temperature-data.zip

## EDA 

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.tools as tls
import seaborn as sns


In [ ]:
global_temp_df = pd.read_csv('./temperature-data/GlobalTemperatures.csv', parse_dates=['dt'])

want to see the first rows of the df --> df.head()
<br>
want to see the last rows of the df --> df.tails()

In [ ]:
global_temp_df.___()

In [ ]:
global_temp_df.___()

want to know more info?
<br>
df.info() function could help!

In [ ]:
global_temp_df.___()

Are you afraid of uncleaned data? missing values could be your one of biggest nightmares in data cleaning
<br>
so, go and count them!
<br>
isna() --> indicates if the values are NA

In [ ]:
global_temp_df.___().sum()

Now, generate some descriptive statistics
<br>
df.describe() could help

In [ ]:
global_temp_df.___()

What is the minimum/maximum date in our dataset?

In [ ]:
___(global_temp_df['dt'])

In [ ]:
___(global_temp_df['dt'])

In [ ]:
global_temp_df_indexed = global_temp_df.set_index('dt')

In [ ]:
global_temp_df_indexed

In [ ]:
global_temp_df_indexed[['LandAverageTemperature']].plot(figsize=(10,6))

In [ ]:
sp_year = global_temp_df[global_temp_df['dt'].dt.year == 2015].set_index('dt')
sp_year[['LandAverageTemperature']].plot()

from your observations, what's the hottest month in 2015?

In [ ]:
years = global_temp_df['dt'].dt.year.unique()

In [ ]:
mean_temp_world = []

for year in years:
    mean_temp_world.append(global_temp_df[global_temp_df['dt'].apply(
        lambda x: x.year) == year]['LandAverageTemperature'].mean())

In [ ]:
d = {'Year': years, 'Land Avg. Temperature Per Year': mean_temp_world}

In [ ]:
pd.DataFrame(data=d).set_index('Year').plot()

Do you see a trend? what's it?

In [ ]:
temp_bycountry_df = pd.read_csv('./temperature-data/GlobalLandTemperaturesByCountry.csv', parse_dates=['dt'])
temp_bycountry_df

In [ ]:
temp_bycountry_df.info()

In [ ]:
temp_bycountry_df.isna().sum()

In [ ]:
temp_bycountry_df['Country'].unique()

In [ ]:
temp_bycountry_df

In [ ]:
max(temp_bycountry_df['dt'])

here, domain knowledge helped us in cleaning! 👍

In [ ]:
temp_bycountry_df = temp_bycountry_df[temp_bycountry_df['dt'].dt.year >= 1825]

In [ ]:
temp_bycountry_df['year'] = temp_bycountry_df['dt'].dt.year

In [ ]:
temp_bycountry_df.tail()

for each country get the avg temp in the whole year.. so we need a group of country and year, right?

In [ ]:
grouping_country_by_year = temp_bycountry_df.___(by=['Country', 'year'])['AverageTemperature'].mean().reset_index()

In [ ]:
grouping_country_by_year

In [ ]:
grouping_country_by_year[grouping_country_by_year['Country']=='Egypt']
#.set_index('year')[['AverageTemperature']].plot()

In [ ]:
codes = pd.read_csv('./temperature-data/mapping.csv')
codes

In [ ]:
codes.loc[codes.name=='Åland Islands', 'name'] = 'Åland'

In [ ]:
codes

In [ ]:
codes['Country'] = codes['name']

In [ ]:
len(codes['name'].unique())

Power of collecting data from different sources and integrate them! ⭐

In [ ]:
data = pd.merge(left = grouping_country_by_year, right = codes, on = 'Country', how = 'left')

In [ ]:
data.head()

In [ ]:
len(grouping_country_by_year['Country'].unique())

In [ ]:
map_df = data[data['year']==2013]
map_df

In [ ]:
import plotly.express as px

In [ ]:
fig = go.Figure(
    data=go.Choropleth(
    locations = map_df['alpha-3'],
    z = map_df['AverageTemperature'],
    text = map_df['Country'],
    colorscale = ('#A93226', '#E67E22', '#F5B041', '#3498DB', '#2874A6', '#283747'),
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '°C',
    colorbar_title = 'AverageTemperature',
)
    )

fig.update_layout(
    title_text='2013 AverageTemperature',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text = '2013 Earth AverageTemperature',
        showarrow = False
    )]
)

fig.show()

In [ ]:
data

dropna is another way of cleaning missing values.

In [ ]:
countries = data.__(axis = 0).groupby(by = ['region', 'Country', 'year']).mean().reset_index()

In [ ]:
countries.head()

In [ ]:
mean = countries.groupby(['Country', 'region'])['AverageTemperature'].mean().reset_index()
maximum = countries.groupby(['Country', 'region'])['AverageTemperature'].max().reset_index()

In [ ]:
difference = pd.merge(left = mean, right = maximum, on = ['Country', 'region'])
difference['diff'] = difference['AverageTemperature_y'] - difference['AverageTemperature_x']


In [ ]:
fig = go.Figure()
fig.update_layout(
                  title="Difference in Temperature (Countries)", title_font_size = 20,
                  font=dict( family="Courier New, monospace", size=13,color="#7f7f7f"),
                  template = "ggplot2", autosize = False, height = 3500, width = 750)

fig.update_xaxes(showline=True, linewidth=1, linecolor='gray')
fig.update_yaxes(showline=True, linewidth=1, linecolor='gray')

sort_diff = difference[['Country', 'region', 'diff']].sort_values(by = 'diff', ascending = True)
fig.add_trace(go.Bar(x = sort_diff['diff'], y = sort_diff['Country'], orientation = 'h',
                    marker=dict(color='rgb(222,184,135)', line=dict( color='rgb(188,143,143)', width=0.6))))
fig.show()

Africa, despite its low contribution to greenhouse gas emissions, remains the most vulnerable continent. Africa is the most vulnerable continent to climate change impacts under all climate scenarios above 1.5 degrees Celsius.
<br>
resource: https://www.afdb.org/en/cop25/climate-change-africa 